<a href="https://colab.research.google.com/github/SirawatVry/ai-ocr-translate/blob/main/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_ocr_%2B_translate_by_ai_for_thai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install PyMuPDF
%pip install pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 60.5 MB/s eta 0:00:00


In [ ]:
import time
from PIL import Image
import pytesseract
from io import BytesIO
import requests
import json
import fitz
API_KEY = "5QnUpqx5JD4n2hVyXNBUfMFYm54VAMzY"

def ocr_image(image_bytes, retries=3):
    url = "https://api.aiforthai.in.th/ocr"
    headers = {"Apikey": API_KEY}
    files = {
        'file': ('page.png', image_bytes, 'image/png')
    }

    for attempt in range(retries):
        try:
            response = requests.post(url, headers=headers, files=files, timeout=20)
            if response.status_code == 200:
                result = response.json()
                return result.get("originaltext", "")
            else:
                print(f"OCR API error: Status {response.status_code}")
                print("Response body:", response.text)
        except Exception as e:
            print(f"OCR Exception (attempt {attempt + 1}):", e)
        time.sleep(2)

    print("Fallback to pytesseract OCR")
    try:
        img = Image.open(BytesIO(image_bytes))
        text = pytesseract.image_to_string(img, lang='eng')
        return text
    except Exception as e:
        print("Fallback OCR failed:", e)
        return ""

def translate_en_to_th(text):
    url = "https://api.aiforthai.in.th/xiaofan-en-th/en2th"
    headers = {
        "apikey": API_KEY,
        "Content-Type": "application/json"
    }
    payload = json.dumps({"text": text})

    try:
        response = requests.post(url, headers=headers, data=payload)
        if response.status_code != 200:
            print(f"Error during Translation API call: Status Code {response.status_code}")
            print(f"Response body: {response.text}")
            return ""
        result = response.json()
        return result.get("translated_text", response.text)
    except json.JSONDecodeError:
        print("Error decoding JSON from Translation API response.")
        print(f"Response body: {response.text}")
        return ""

def pdf_to_images(pdf_path):
    images = []
    with fitz.open(pdf_path) as doc:
        for page_num, page in enumerate(doc):
            pix = page.get_pixmap(dpi=200)
            img_bytes = pix.tobytes("png")
            images.append((page_num + 1, img_bytes))
    return images

def process_pdf(file_path):
    images = pdf_to_images(file_path)

    for page_num, img_bytes in images:
        print(f"\nกำลังประมวลผลหน้า {page_num} ...")
        eng_text = ocr_image(img_bytes)
        if eng_text:
            print(" ข้อความ OCR:", eng_text[:60].replace("\n", " ") + "...")
            thai_text = translate_en_to_th(eng_text)
            print(" แปลแล้ว:", thai_text.strip() + "\n")

            print(f"\n=== แปลหน้า {page_num} ===\n{thai_text.strip()}\n")
        else:
            print("ไม่สามารถดึงข้อความจากหน้าได้\n")

    print("\n เสร็จสิ้น! แสดงผลทั้งหมดแล้วทางหน้าจอ")

# ---- เรียกใช้งาน ----
if __name__ == "__main__":
    pdf_path = "/content/drive/MyDrive/Colab Notebooks/dataset/os sheet_rotated.pdf"
    process_pdf(pdf_path)


กำลังประมวลผลหน้า 1 ...
OCR API error: Status 502
Response body: {
  "message":"An invalid response was received from the upstream server",
  "request_id":"371791103369a3575f42d109631ea372"
}
OCR API error: Status 502
Response body: {
  "message":"An invalid response was received from the upstream server",
  "request_id":"e5a42d5c4ed64c61e8420b12ede7fe09"
}
OCR API error: Status 502
Response body: An invalid response was received from the upstream server

Fallback to pytesseract OCR
 ข้อความ OCR: a aan’  Computer System Structure  O Computer system can be ...
 แปลแล้ว: โครงสร้างระบบคอมพิวเตอร์ของ Aan's ระบบคอมพิวเตอร์ O สามารถแบ่งออกเป็น 4 องค์ประกอบได้: ฮาร์ดแวร์ — ให้ทรัพยากรการคอมพิวเตอร์ขั้นพื้นฐาน » ระบบความทรงจำ ไอ / โอ อุปกรณ์ปฏิบัติการ » การควบคุมและพิกัดการใช้ฮาร์ดแวร์ในโปรแกรมแอปพลิเคชันแอพพลิเคชันและแอปพลิเคชันผู้ใช้งานต่าง ๆ — กำหนดวิธีการที่ทรัพยากรระบบใช้เพื่อแก้ปัญหาการคอมพิวเตอร์ของผู้ใช้ » เครื่องประกอบคำ ประมวลผล ผู้ใช้เว็บ ระบบฐานข้อมูล ผู้ใช้งานวิดีโอเกม » เครื่องจ